<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SPSS Collaboration and Deployment Services

### Compatible with Cloud Pak for Data Only

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
 - Setup
 - Binding machine learning engine
 - Subscriptions
 - Performance monitor, scoring and payload logging
 - Quality monitor and feedback logging
 - Fairness,Drift monitoring and explanations

## Setup

### Sample model creation using SPSS Modeler

- Download training data set from [here](https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/credit_risk_training.csv)
- Download SPSS Modeler stream from [here](https://github.com/IBM/watson-openscale-samples/tree/main/assets/models/credit_risk/german_credit_risk_tutorial.str)
- Deploy the model using SPSS C&DS as web service

### Installation and authentication

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
!pip install --upgrade ibm-watson-openscale | tail -n 1

Import and initiate.

In [86]:
WOS_CREDENTIALS = {
    "url": "https://namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com",
    "username": "admin",
    "password": "password"
}

In [87]:
WML_CREDENTIALS = {
                   "url": "https://namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com",
                    "username": "admin",
                    "password": "password",
                    "instance_id": "wml_local",
                   "version" : "3.5"

                  }

In [88]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_API_KEY_ID = "917Z-0MQzVpgdqkHClXbfDchrXZ_bl7kbCxZSkynzsLP"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/e0b56432b1f1bd804706dc29b8a89ca1:57b5eb6e-7b5d-4b90-a8e8-3736129c9010::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "https://s3.us.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [89]:
BUCKET_NAME = "wmlv4-donotdelete-pr-e8rgr9bhcjtlae" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [1]:
instance_id='00000000-0000-0000-0000-1604128772718153' ## default instance ID: 00000000-0000-0000-0000-0000000000000000

In [147]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],service_instance_id=instance_id,authenticator=authenticator)
wos_client.version


/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'3.0.2'

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS (ICP)
- DATABASE_CREDENTIALS (DB2 on ICP)
- SCHEMA_NAME

In [148]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [149]:
SCHEMA_NAME = 'SPSSTF01'

### DataMart setup

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were **not** supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there **unless** there is an existing datamart **and** the **KEEP_MY_INTERNAL_POSTGRES** variable is set to **True**. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [150]:
wos_client.data_marts.show()

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


None,None,False,active,2020-10-31 07:22:08.779000+00:00,00000000-0000-0000-0000-1604128772718153


In [151]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 00000000-0000-0000-0000-1604128772718153


/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a id="binding"></a>
## Bind machine learning engines

### Bind  `SPSS C&DS` machine learning engine

Provide credentials using following fields:
- `username`
- `password`
- `url`

In [183]:
SPSS_CDS_ENGINE_CREDENTIALS = {
        "url": "http://dismiss1.fyre.ibm.com:9080",
        "username": "admin",
        "password": "spss",
    }

In [184]:
SERVICE_PROVIDER_NAME = "V2 SPSS test"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [185]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Deleted existing service_provider for WML instance: 9469fe28-59d3-4194-9523-9f0d2f7d82b2


/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [186]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.SPSS_COLLABORATION_AND_DEPLOYMENT_SERVICES,
        credentials=SPSSCredentials(
            url=SPSS_CDS_ENGINE_CREDENTIALS['url'],      
            username=SPSS_CDS_ENGINE_CREDENTIALS["username"],
            password=SPSS_CDS_ENGINE_CREDENTIALS['password']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of adding service provider b45a656e-38fc-4a61-87ad-28025a2032b0 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [188]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result['resources']
asset_deployment_details

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


[{'metadata': {'guid': 'credit_risk_db2',
   'url': 'http://dismiss1.fyre.ibm.com:9080/scoring/rest/configuration/credit_risk_db2/score'},
  'entity': {'name': 'credit_risk_db2',
   'type': 'online',
   'scoring_endpoint': {'url': 'http://dismiss1.fyre.ibm.com:9080/scoring/rest/configuration/credit_risk_db2/score'},
   'asset': {'asset_id': '091edbc5c4e74be500000168f14b3eb1be39',
    'url': '/credit_risk/credit_risk_db2.str',
    'name': 'credit_risk_db2'},
   'asset_properties': {'input_data_schema': {'type': 'struct',
     'name': 'DASH101323.CREDIT_RISK_SCORING_DATA',
     'id': 'id4XSTWFFJR3V',
     'fields': [{'name': 'CheckingStatus',
       'type': 'string',
       'nullable': False},
      {'name': 'LoanDuration', 'type': 'long', 'nullable': False},
      {'name': 'CreditHistory', 'type': 'string', 'nullable': False},
      {'name': 'LoanPurpose', 'type': 'string', 'nullable': False},
      {'name': 'LoanAmount', 'type': 'long', 'nullable': False},
      {'name': 'ExistingSavin

In [189]:
MODEL_NAME='german_credit_risk_tutorial_BiasQA' # use the model name here 
model_asset_details_from_deployment = [asset for asset in asset_deployment_details if asset['entity']["name"]==MODEL_NAME]
source_uid = [asset['entity']['asset']['asset_id'] for asset in asset_deployment_details if asset['entity']["name"]==MODEL_NAME]
if len(model_asset_details_from_deployment)>0:
    [model_asset_details_from_deployment] = model_asset_details_from_deployment
    [source_uid] = source_uid
else:
    raise ValueError('Model with name "{}" not found.'.format(MODEL_NAME))

<a id="subsciption"></a>
## Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [218]:
wos_client.subscriptions.show()

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


091edbc5c4e74be5000001686a7c7110cafe,german_credit_risk_tutorial_BiasQA,00000000-0000-0000-0000-1604128772718153,german_credit_risk_tutorial_BiasQA,german_credit_risk_tutorial_BiasQA,b45a656e-38fc-4a61-87ad-28025a2032b0,active,2020-11-01 00:59:15.557000+00:00,05411c64-167c-4eba-b1f4-9934222ec9f0


In [193]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == source_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [221]:
training_data_reference= TrainingDataReference(type='cos',
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL}))

In [223]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=model_asset_details_from_deployment['metadata']['guid'],
            name=model_asset_details_from_deployment['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['$NP-No Risk','$NP-Risk'],
            prediction_field='$N-Risk',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=training_data_reference,
            input_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["input_data_schema"]),
            output_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["output_data_schema"])

        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'69e435b9-b62f-4c81-8cf4-fafd17e19476'

## Performance monitor, scoring and payload logging

### Score the credit risk model and measure response time

In [226]:
import requests
from requests.auth import HTTPBasicAuth
import time
import json

scoring_endpoint = subscription_details.to_dict()['entity']['deployment']['url']
input_table_id = subscription_details.to_dict()['entity']['asset_properties']['input_data_schema']['id']
node_id = subscription_details.to_dict()['entity']['asset']['name']

scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': [{'input': [
            {'name': 'CheckingStatus', 'value': '0_to_200'}, {'name': 'LoanDuration', 'value': 31},
            {'name': 'CreditHistory', 'value': 'credits_paid_to_date'}, {'name': 'LoanPurpose', 'value': 'other'},
            {'name': 'LoanAmount', 'value': 1889}, {'name': 'ExistingSavings', 'value': '100_to_500'},
            {'name': 'EmploymentDuration', 'value': 'less_1'}, {'name': 'InstallmentPercent', 'value': 3},
            {'name': 'Sex', 'value': 'female'}, {'name': 'OthersOnLoan', 'value': 'none'},
            {'name': 'CurrentResidenceDuration', 'value': 3}, {'name': 'OwnsProperty', 'value': 'savings_insurance'},
            {'name': 'Age', 'value': 32}, {'name': 'InstallmentPlans', 'value': 'none'},
            {'name': 'Housing', 'value': 'own'}, {'name': 'ExistingCreditsCount', 'value': 1},
            {'name': 'Job', 'value': 'skilled'}, {'name': 'Dependents', 'value': 1},
            {'name': 'Telephone', 'value': 'none'}, {'name': 'ForeignWorker', 'value': 'yes'}]}]}], 'id': node_id}

start_time = time.time()
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))

response_time = int((time.time() - start_time)*1000)
result = resp_score.json()

print(result)

{'providedBy': 'german_credit_risk_tutorial_BiasQA', 'id': 'c9d4ba4c-91da-4fa2-bacf-4575ff0a5af6', 'columnNames': {'name': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', '$N-Risk', '$NC-Risk', '$NP-No Risk', '$NP-Risk']}, 'rowValues': [{'value': [{'value': '0_to_200'}, {'value': '31'}, {'value': 'credits_paid_to_date'}, {'value': 'other'}, {'value': '1889'}, {'value': '100_to_500'}, {'value': 'less_1'}, {'value': '3'}, {'value': 'female'}, {'value': 'none'}, {'value': '3'}, {'value': 'savings_insurance'}, {'value': '32'}, {'value': 'none'}, {'value': 'own'}, {'value': '1'}, {'value': 'skilled'}, {'value': '1'}, {'value': 'none'}, {'value': 'yes'}, {'value': 'No Risk'}, {'value': '0.8252855725848809'}, {'value': '0

### Store the request and response in payload logging table

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your application so it is logged automatically each time you score the model.

In [227]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Payload data set id:  4af55405-0d25-4d7c-95fe-bf9fb824dd0e


In [3]:
import pandas as pd
df_data = pd.read_csv("https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk_spss/payload_credit_risk.csv")
df_data=df_data.drop(['Risk'],axis=1)

In [4]:
#score using couple sample records
payload=df_data.sample(2)

In [5]:
values = payload.values.tolist()
columns = payload.columns.tolist()

In [6]:
records_list=[]
for i in payload.to_dict('records'):
    b=[{"name":x,"value":v} for x,v in i.items()]
    records_list.append({'input': b})

In [232]:
scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': records_list}], 'id': node_id}

In [233]:
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))

response_time = int((time.time() - start_time)*1000)
result = resp_score.json()
print(result)

{'providedBy': 'german_credit_risk_tutorial_BiasQA', 'id': '4a05154c-9ff4-43eb-b8ce-9d6fbbb87137', 'columnNames': {'name': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', '$N-Risk', '$NC-Risk', '$NP-No Risk', '$NP-Risk']}, 'rowValues': [{'value': [{'value': '0_to_200'}, {'value': '13'}, {'value': 'prior_payments_delayed'}, {'value': 'repairs'}, {'value': '1889'}, {'value': 'greater_1000'}, {'value': '4_to_7'}, {'value': '2'}, {'value': 'male'}, {'value': 'none'}, {'value': '2'}, {'value': 'savings_insurance'}, {'value': '34'}, {'value': 'stores'}, {'value': 'own'}, {'value': '1'}, {'value': 'skilled'}, {'value': '2'}, {'value': 'none'}, {'value': 'yes'}, {'value': 'No Risk'}, {'value': '0.7983974913362735'}, {'valu

### Format scoring response for payload logging

In [234]:
res_values=[]
for i in result['rowValues']:
    d= [j for j in i['value']]
    res_values.append([k['value'] for k in d])

In [235]:
dtype_idx=[1,4,7,10,12,15,17] # change numeric features values in scoring response from String to Integer
dtype_predictions_idx=[21,22,23] # change prediction, probability column values in scoring response from String to Float

for val in res_values:
    for idx in dtype_idx:
        val[idx]=int(val[idx])
    for idx in dtype_predictions_idx:
        val[idx]=float(val[idx])     

[['0_to_200',
  13,
  'prior_payments_delayed',
  'repairs',
  1889,
  'greater_1000',
  '4_to_7',
  2,
  'male',
  'none',
  2,
  'savings_insurance',
  34,
  'stores',
  'own',
  1,
  'skilled',
  2,
  'none',
  'yes',
  'No Risk',
  0.7983974913362735,
  0.7983974913362735,
  0.20160250866372653],
 ['greater_200',
  4,
  'credits_paid_to_date',
  'other',
  3109,
  'greater_1000',
  'less_1',
  4,
  'female',
  'none',
  3,
  'real_estate',
  59,
  'none',
  'free',
  1,
  'management_self-employed',
  1,
  'none',
  'yes',
  'No Risk',
  0.8873499973689839,
  0.8873499973689839,
  0.11265000263101611]]

In [237]:
request = {
            "fields": payload.columns.tolist(),
            "values": payload.values.tolist()
        }
response = {
            "fields": result['columnNames']['name'],
            "values": res_values
        }

In [239]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
               scoring_id=str(uuid.uuid4()),
               request=request,
               response=response,
               response_time=460
           )])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Number of records in the payload logging table: 4


In [240]:
wos_client.data_sets.show_records(payload_data_set_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


None,1889,270a514d-d428-41e5-a862-ccf570d58602-1,2,greater_1000,2020-11-01T01:19:33.326Z,0.7983974913362735,"[0.7983974913362735, 0.20160250866372653]",0_to_200,repairs,skilled,0.20160250866372653,stores,none,0.7983974913362735,0.7983974913362735,13,34,prior_payments_delayed,2,yes,own,No Risk,1,german_credit_risk_tutorial_BiasQA,none,male,2,savings_insurance,4_to_7
None,3109,270a514d-d428-41e5-a862-ccf570d58602-2,4,greater_1000,2020-11-01T01:19:33.326Z,0.8873499973689839,"[0.8873499973689839, 0.11265000263101611]",greater_200,other,management_self-employed,0.11265000263101611,none,none,0.8873499973689839,0.8873499973689839,4,59,credits_paid_to_date,3,yes,free,No Risk,1,german_credit_risk_tutorial_BiasQA,none,female,1,real_estate,less_1
None,1889,06f651d2-96a8-43ac-a0b7-26fc86253c99-1,2,greater_1000,2020-11-01T01:19:14.361Z,0.7983974913362735,"[0.7983974913362735, 0.20160250866372653]",0_to_200,repairs,skilled,0.20160250866372653,stores,none,0.7983974913362735,0.7983974913362735,13,34,prior_payments_delayed,2,yes,own,No Risk,1,german_credit_risk_tutorial_BiasQA,none,male,2,savings_insurance,4_to_7
None,3109,06f651d2-96a8-43ac-a0b7-26fc86253c99-2,4,greater_1000,2020-11-01T01:19:14.361Z,0.8873499973689839,"[0.8873499973689839, 0.11265000263101611]",greater_200,other,management_self-employed,0.11265000263101611,none,none,0.8873499973689839,0.8873499973689839,4,59,credits_paid_to_date,3,yes,free,No Risk,1,german_credit_risk_tutorial_BiasQA,none,female,1,real_estate,less_1


## Quality monitor and feedback logging

### Enable quality monitoring

In [241]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitor instance creation 87163786-2bc5-4e6b-a6b4-1019840ff6e7 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [242]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'87163786-2bc5-4e6b-a6b4-1019840ff6e7'

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

In [243]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


#### Store feedback using CSV format from file

In [8]:
#!wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_feedback.csv
feed_data_load = pd.read_csv('https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk_spss/feedback_credit_risk.csv')
feedback_data = json.loads(feed_data_load.to_json(orient='records'))
feedback_data

[{'CheckingStatus': 'no_checking',
  'LoanDuration': 31,
  'CreditHistory': 'prior_payments_delayed',
  'LoanPurpose': 'other',
  'LoanAmount': 4066,
  'ExistingSavings': '100_to_500',
  'EmploymentDuration': '4_to_7',
  'InstallmentPercent': 3,
  'Sex': 'male',
  'OthersOnLoan': 'none',
  'CurrentResidenceDuration': 2,
  'OwnsProperty': 'unknown',
  'Age': 56,
  'InstallmentPlans': 'stores',
  'Housing': 'free',
  'ExistingCreditsCount': 2,
  'Job': 'skilled',
  'Dependents': 1,
  'Telephone': 'yes',
  'ForeignWorker': 'yes',
  'Risk': 'Risk'},
 {'CheckingStatus': '0_to_200',
  'LoanDuration': 20,
  'CreditHistory': 'credits_paid_to_date',
  'LoanPurpose': 'repairs',
  'LoanAmount': 1481,
  'ExistingSavings': 'greater_1000',
  'EmploymentDuration': 'less_1',
  'InstallmentPercent': 3,
  'Sex': 'male',
  'OthersOnLoan': 'none',
  'CurrentResidenceDuration': 3,
  'OwnsProperty': 'savings_insurance',
  'Age': 58,
  'InstallmentPlans': 'stores',
  'Housing': 'own',
  'ExistingCreditsCount

In [245]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of storing records with request id: 6fc25b95-78bf-4d4d-afb8-34ff697869ab 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [216]:
wos_client.data_sets.show_records(data_set_id=feedback_dataset_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


01026699-c677-4db5-8c18-c7c0244ece4e,2020-11-01T00:56:27.980Z,1676,2,100_to_500,0_to_200,appliances,management_self-employed,stores,none,39,None,43,outstanding_credit,4,yes,free,2,yes,female,2,savings_insurance,No Risk,less_1
06e260d6-a2e2-4daf-88a2-db059ed0aeed,2020-11-01T00:56:27.980Z,2067,3,greater_1000,greater_200,appliances,skilled,none,none,36,None,50,credits_paid_to_date,4,yes,rent,2,none,male,1,real_estate,No Risk,1_to_4
5d9f8745-0fd1-4d56-9ac1-01e685502937,2020-11-01T00:56:27.980Z,2740,2,less_100,0_to_200,other,skilled,stores,none,37,None,44,prior_payments_delayed,4,yes,rent,2,yes,female,1,savings_insurance,No Risk,greater_7
64360f01-e70e-4f81-99e0-ae35e45c14c6,2020-11-01T00:56:27.980Z,4190,4,less_100,less_0,education,skilled,none,none,15,None,50,outstanding_credit,3,yes,own,1,none,female,1,unknown,No Risk,less_1
899b1680-8174-4f34-a49e-84f6856af743,2020-11-01T00:56:27.980Z,1735,3,100_to_500,greater_200,furniture,skilled,none,none,22,None,32,outstanding_credit,4,yes,own,1,yes,female,1,savings_insurance,No Risk,less_1
94d4cdc4-82be-4b06-a188-dd6cdf29da1c,2020-11-01T00:56:27.980Z,4763,2,less_100,0_to_200,repairs,skilled,none,none,34,None,57,prior_payments_delayed,2,yes,rent,1,none,male,2,unknown,No Risk,less_1
9d398ada-6031-45b7-8310-07ba707910c7,2020-11-01T00:56:27.980Z,2661,5,greater_1000,0_to_200,car_used,skilled,none,none,33,None,21,prior_payments_delayed,3,yes,own,1,yes,male,1,real_estate,Risk,4_to_7
a82282a7-23f3-4dcb-bba5-1af8305adb7b,2020-11-01T00:56:27.980Z,1257,2,100_to_500,greater_200,vacation,management_self-employed,stores,none,38,None,55,credits_paid_to_date,2,yes,own,2,yes,male,2,real_estate,No Risk,greater_7
ab447e7e-aed4-44e0-b9d0-f343e18ab9ef,2020-11-01T00:56:27.980Z,1388,2,less_100,less_0,appliances,management_self-employed,none,none,36,None,28,prior_payments_delayed,2,yes,free,1,none,male,2,unknown,Risk,4_to_7
ac66869b-b7fd-48d6-b4ba-003a22b915b4,2020-11-01T00:56:27.980Z,2460,3,100_to_500,less_0,vacation,management_self-employed,stores,none,39,None,44,outstanding_credit,4,yes,own,1,yes,male,2,real_estate,No Risk,1_to_4


### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [246]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitoring run 7928babb-0d52-45f0-a74f-06e792cd3b6a 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



running

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



finished

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------
 Successfully finished run 
---------------------------




In [247]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


## Fairness monitoring and explanations

### Enable and run fairness monitoring

In [248]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 40
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitor instance creation 7cc73e82-b1b4-4c40-9c5b-eb05e46723a0 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



preparing

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'7cc73e82-b1b4-4c40-9c5b-eb05e46723a0'

### Score, format and store payload records

In [249]:
payload=df_data.sample(50)

In [250]:
records_list=[]
for i in payload.to_dict('records'):
    b=[{"name":x,"value":v} for x,v in i.items()]
    records_list.append({'input': b})

In [251]:
scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': records_list}], 'id': node_id}
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))
result = resp_score.json()

In [252]:
res_values=[]
for i in result['rowValues']:
    d= [j for j in i['value']]
    res_values.append([k['value'] for k in d])

In [253]:
dtype_idx=[1,4,7,10,12,15,17] # change numeric features values in scoring response from String to Integer
dtype_predictions_idx=[21,22,23] # change prediction, probability column values in scoring response from String to Float

for val in res_values:
    for idx in dtype_idx:
        val[idx]=int(val[idx])
    for idx in dtype_predictions_idx:
        val[idx]=float(val[idx])
len(res_values)       

50

In [254]:
request = {
            "fields": payload.columns.tolist(),
            "values": payload.values.tolist()
        }
response = {
            "fields": result['columnNames']['name'],
            "values": res_values
        }

In [255]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
               scoring_id=str(uuid.uuid4()),
               request=request,
               response=response,
               response_time=460
           )])
time.sleep(10)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Number of records in the payload logging table: 54


In [256]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitoring run 3ceecad6-bc94-48fe-afd6-bcf9dddba5b6 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



running

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

.

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



finished

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------
 Successfully finished run 
---------------------------




In [257]:
time.sleep(10)
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongl

### Explainability configuration and run

#### Enable explainability

In [258]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitor instance creation d6e3fb61-910b-43d5-bc0c-defdf4ad0a7a 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



preparing

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'd6e3fb61-910b-43d5-bc0c-defdf4ad0a7a'

#### Get sample transaction_id from payload logging table (`scoring_id`)

In [259]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Running explanations on scoring IDs: ['d9530502-b301-4eb1-9297-e9eb7fa85b30-1']


/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{
  "metadata": {
    "explanation_task_ids": [
      "11280990-24ff-452b-90c6-78c0e1d59404"
    ],
    "created_by": "1000330999",
    "created_at": "2020-11-01T01:24:11.491618Z"
  }
}


'11280990-24ff-452b-90c6-78c0e1d59404'

In [264]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{'metadata': {'explanation_task_id': '11280990-24ff-452b-90c6-78c0e1d59404',
  'created_by': '1000330999',
  'created_at': '2020-11-01T01:24:11.491618Z',
  'updated_at': '2020-11-01T01:24:44.328198Z'},
 'entity': {'status': {'state': 'in_progress'},
  'asset': {'id': '091edbc5c4e74be5000001686a7c7110cafe',
   'name': 'german_credit_risk_tutorial_BiasQA',
   'input_data_type': 'structured',
   'problem_type': 'binary',
   'deployment': {'id': 'german_credit_risk_tutorial_BiasQA',
    'name': 'german_credit_risk_tutorial_BiasQA'}},
  'input_features': [{'name': 'CheckingStatus',
    'value': 'greater_200',
    'feature_type': 'categorical'},
   {'name': 'LoanDuration', 'value': '16', 'feature_type': 'numerical'},
   {'name': 'CreditHistory',
    'value': 'outstanding_credit',
    'feature_type': 'categorical'},
   {'name': 'LoanPurpose', 'value': 'vacation', 'feature_type': 'categorical'},
   {'name': 'LoanAmount', 'value': '4553', 'feature_type': 'numerical'},
   {'name': 'ExistingSavin

### Enable and run drift monitoring

In [9]:
!rm -rf creditrisk_spss_drift_detection_model.tar.gz
!wget -O creditrisk_spss_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/spss_creditrisk_drift_detection_model.tar.gz?raw=true 

--2020-11-04 14:33:50--  https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/spss_creditrisk_drift_detection_model.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/spss_creditrisk_drift_detection_model.tar.gz [following]
--2020-11-04 14:33:51--  https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/spss_creditrisk_drift_detection_model.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/models/credit_risk/spss_creditrisk_drift_detection_model.tar.gz [following]
--2020-11-04 14:33:51--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/models

In [265]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_spss_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [266]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [267]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 40,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitor instance creation 915ae115-35d6-46b7-8c0a-96921e27bcc2 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



active

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'915ae115-35d6-46b7-8c0a-96921e27bcc2'

In [268]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,





 Waiting for end of monitoring run bace9cfb-b55c-4685-b4cc-fa4bcd34df2e 





/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



running

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



finished

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,




---------------------------
 Successfully finished run 
---------------------------




In [269]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'namespace1-cpd-namespace1.apps.islnov09.os.fyre.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit SPSS model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---